In [ ]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/WELFake_Dataset.csv')

In [4]:
df_clean = df.copy()
df_clean.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


### Data Cleaning and Preprocessing

#### 1- data cleaning

In [8]:
# quick review of rows count and class distribution
print("initial rows:", df_clean.shape[0])
print("\nclass distribution:\n", df_clean['label'].value_counts())
print("\nclass balance(percent): \n", df_clean['label'].value_counts(normalize=True)*100)

initial rows: 72134

class distribution:
 label
1    37106
0    35028
Name: count, dtype: int64

class balance(percent): 
 label
1    51.440375
0    48.559625
Name: proportion, dtype: float64


In [14]:
# remove duplicates
before = df_clean.shape[0]
df_clean = df_clean.drop_duplicates(subset=['text'])
after = df_clean.shape[0]
print(f"\nremoved {before - after} duplicate rows")


removed 9415 duplicate rows


In [15]:
# missing values review
print("\n missing values: \n", df_clean[['title', 'text', 'label']].isnull().sum())




 missing values: 
 title    518
text       1
label      0
dtype: int64


In [16]:
# drop missing values for text column, no text no model
df_clean = df_clean.dropna(subset=['text']).reset_index(drop=True)

#fill missing titles with empty string
df_clean['title'] = df_clean['title'].fillna('')

#### 2- add helpful diagnostics

In [17]:
# add text len and word count diagnostics
df_clean['text'] = df_clean['text'].astype(str)
df_clean ['text_len'] = df_clean['text'].apply(len)
df_clean['words_count'] = df_clean['text'].apply(lambda x: len(x.split()))

In [18]:
df_clean[['text_len','words_count']].describe()

,text_len,words_count
count,62718.000000,62718.000000
mean,3316.446746,548.983641
std,3631.818948,610.055911
min,1.000000,0.000000
25%,1477.000000,244.000000
50%,2480.000000,406.000000
75%,4131.750000,679.000000
max,142961.000000,24234.000000


In [21]:
# show top 10 longest texts (by characters)
longest_idx = df_clean['text_len'].nlargest(10).index
for i in longest_idx:
    print("Index:", i, "Length:", df_clean.loc[i,'text_len'])
    print(df_clean.loc[i,'text'][:1000])   # print first 1000 chars for quick check
    print('-'*80)


Index: 6298 Length: 142961
Заседание Международного дискуссионного клуба «Валдай» Владимир В. Путин Сеть Вольтер | Сочи (Россия) | 27 октября 2016 English français Т.Колтон (как переведено): Добрый день, дамы и господа! Меня зовут Тимоти Колтон. Я знаком со многими присутствующими сегодня здесь. Мне чрезвычайно приятно, что меня попросили выступить модератором этой последней сессии в 2016 году в рамках Валдайского дискуссионного клуба, так мы его называем.
Хотел бы особо приветствовать нашего основного выступающего – это Владимир Владимирович Путин, Президент Российской Федерации. Он нашёл время, чтобы приехать сюда, принять участие. Мы знаем, как он занят. Мы также высоко ценим его готовность отвечать на наши вопросы.
В какой‑то момент, когда Вы выйдете на пенсию, господин Президент, и напишете свои мемуары, попытайтесь подумать, сколько времени Вы потратили на ответы и вопросы. Я уверен, Вы будете поражены этим. Мы очень благодарим Вас за это. Спасибо за то, что Вы с нами сегодня.
По

the long texts look normal, but we may have some news with diffrent language, best choice for now is to delete them, but if we want to buils a multilingual model we can keep them

### Detect the number of Languages

In [23]:
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0  # for consistent results

In [38]:
# Detect language for each text
def safe_detect(text):
    try:
        text = str(text).strip()
        if not text:  # empty string
            return 'unknown'
        return detect(text)
    except Exception as e:
        return 'unknown'

df_clean['lang'] = df_clean['text'].apply(safe_detect)

In [42]:
df_clean['lang'].value_counts()


lang
en         62119
ru           156
es           142
de           106
unknown       58
fr            39
ar            19
pt            11
sw             9
tr             8
it             6
pl             5
so             5
ro             4
nl             4
no             4
hr             3
cy             2
el             2
id             2
vi             2
fi             2
da             1
tl             1
af             1
et             1
zh-cn          1
hu             1
ca             1
sq             1
sv             1
lt             1
Name: count, dtype: int64

In [45]:
# Keep only English texts
df_clean = df_clean[df_clean['lang'] == 'en'].reset_index(drop=True)
print(f"Remaining rows after keeping English only: {df_clean.shape[0]}")


Remaining rows after keeping English only: 62119


In [46]:
df_clean.to_csv('data/WELFake_Dataset_Cleaned.csv', index=False)
print("Cleaned dataset saved to 'data/WELFake_Dataset_Cleaned.csv'")

Cleaned dataset saved to 'data/WELFake_Dataset_Cleaned.csv'
